In [1]:
import numpy as np
import pandas as pd
from pandas import json_normalize
import seaborn as sns
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
import requests 
import time
import re

# pip install geopy
from geopy.geocoders import Nominatim

import warnings
warnings.filterwarnings("ignore")

from geopy_functions import *
from my_functions import *

In [2]:
# import the dataframes
df_uk_masters = pd.read_csv('Datasets/df_uk_masters.csv')                         # all the albums from the UK
df_us_masters = pd.read_csv('Datasets/df_us_masters.csv')                         # albums from the US until 1996, 1998 and 2000
df_us_new_masters = pd.read_csv('Datasets/df_us_new_masters.csv')                         # albums from the US from 1997, 1999 and 2001
df_us_new_masters_clean = pd.read_csv('Datasets/df_us_new_masters_clean.csv')             # albums from the US from 1997, 1999 and 2001, cleaned, merged with df_ratings_20
df_ratings_20 = pd.read_csv('Datasets/df_ratings_20.csv', keep_default_na=False)  # albums with >= 20 votes, mostly from rock, worldwide
df_masters_blended = pd.read_csv('Datasets/df_masters_blended.csv')               # albums from the UK and US (and others) with >= 20 votes until 2000 aprox
df_artists_origins_coordinates = pd.read_csv('Datasets/df_artists_origins_coordinates.csv')               # merge of df_masters_blended with their locations

# print information
print(f'{df_uk_masters.shape}: df_uk_masters')
print(f'{df_us_masters.shape}: df_us_masters')
print(f'{df_us_new_masters.shape}: df_us_new_masters')
print(f'{df_us_new_masters_clean.shape}: df_us_new_masters_clean')
print(f'{df_ratings_20.shape}: df_ratings_20')
print(f'{df_masters_blended.shape}: df_masters_blended')

(56660, 13): df_uk_masters
(48690, 13): df_us_masters
(74652, 13): df_us_new_masters
(6708, 13): df_us_new_masters_clean
(51222, 5): df_ratings_20
(9667, 13): df_masters_blended


In [5]:
df_masters_blended.head(1)

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile
0,15152368,3747909,31909420,NaN,As Living Arrows,Hope and Ruin,2024,0.0,8,"['LP', 'Album']",['Rock'],['Post-Hardcore'],"Post-screamo band from Brighton, UK\r\n"


In [ ]:
df_us_new_masters_clean.head(1)

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile
0,1441645,396963,3314361,Europe,Devin Townsend Project,Contain Us,2011,1177.03,219,['Album'],"['Electronic', 'Rock', 'Pop']","['Alternative Rock', 'Industrial', 'Prog Rock'...",Rock/metal project of [a251249]. It was founde...


# Locations Wikipedia scraping

## Import dataframes

In [9]:
df = pd.read_csv('Datasets/df_rock_ratings.csv', keep_default_na=False)

print(f"{df['artist'].nunique()} artists")
print(f"{df.shape[0]} albums")
print(f"Average of {round(df.shape[0] / df['artist'].nunique(), 2)} albums per artist in the subset with the (mostly UK) albums with more than 10 votes")

FileNotFoundError: [Errno 2] No such file or directory: 'Datasets/df_rock_ratings.csv'

In [9]:
df_20 = pd.read_csv('Datasets/df_rock_ratings_20.csv', keep_default_na=False)

print(f"{df_20['artist'].nunique()} artists")
print(f"{df_20.shape[0]} albums")
print(f"Average of {round(df_20.shape[0] / df_20['artist'].nunique(), 2)} albums per artist in the subset with the (mostly UK) albums with more than 20 votes")

3823 artists
9380 albums
Average of 2.45 albums per artist in the subset with the (mostly UK) albums with more than 20 votes


In [ ]:
df_ratings_20 = pd.read_csv('Datasets/df_ratings_20.csv')
df_ratings_20.shape

(51252, 5)

## Testing

In [28]:
df.sample(5)

,artist,title,year,album_length,tracks,genres,styles,release_country,artist_profile
10580,The 1975,A Brief Inquiry into Online Relationships,2018,58.43,15,"['Rock', 'Pop']","['Indie Rock', 'Alternative Rock', 'Indie Pop']",UK & Europe,British indie rock band. \r\n\r\nPop-rock band...
9889,Le Butcherettes,A Raw Youth,2015,0.00,12,['Rock'],[],US,Formed by Teri Gender Bender and Auryn Jolene ...
6394,John Fogerty,Centerfield,1985,35.33,9,['Rock'],"['Pop Rock', 'Folk Rock', 'Country Rock']",US,"American musician, songwriter, and guitarist (..."
4498,L7,The Beauty Process: Triple Platinum,1997,41.57,12,['Rock'],"['Punk', 'Grunge']",US,American grunge punk/alternative rock band fro...
5742,The Fall,Are You Are Missing Winner,2001,47.68,10,['Rock'],"['Garage Rock', 'Punk', 'Rockabilly']",UK,"Post-punk band from Greater Manchester, UK. 19..."


In [125]:
artists = df['artist'].unique()
len(artists)

9616

In [106]:
artists[4155]

'Life at These Speeds'

In [116]:
artist = "AABSINTHE"
name_changed = artist.title().replace(' ', '_')
name_changed

'Aabsinthe'

In [658]:
artist = "John Grant"
name_changed = artist.replace(' ', '_')
name_changed_band = artist.replace(' ', '_') + ('_(band)')
name_changed_musician = name_changed + ('_(musician)')

url = f"https://en.wikipedia.org/wiki/{name_changed_musician}"
response = requests.get(url).content
soup = BeautifulSoup(response, "html.parser")
table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

try:
    text = table[0].text

    # Step 1: Extract the part after 'Born'
    after_born = text.split("Born", 1)[1]

    text_age = re.search("aged", after_born)

    if text_age:
        # This means the musician is dead
        location = re.split(r'(19\d{2})', after_born)[4].split('Died')[0].strip()
    else:
        try:
            text = re.split(r'(19\d{2})', after_born)[4].split(')')[1]

            if "Other\xa0names" in text:
                location = text.split('Other\xa0names')[0]
            else:
                if "Citizenship" in text:
                    location = text.split('Citizenship')[0]
                else:
                    if "Genres" in text:
                        location = text.split('Genres')[0]
                        print('Genres')
                    else:
                        if "Occupations" in text:
                            location = text.split('Occupations')[0]
                            print('Occupations')
                        else:
                            location = np.nan
        except:  
            location = np.nan
except:
    print('fuck')


Genres


In [659]:
location

'Buchanan, Michigan, U.S.'

In [652]:
name_changed_musician

'John_Grant_(musician)'

In [533]:
artist = "Stone Gossard"
name_changed = artist.replace(' ', '_')
name_changed_band = artist.replace(' ', '_') + ('_(band)')

url = f"https://en.wikipedia.org/wiki/{name_changed}"
response = requests.get(url).content
soup = BeautifulSoup(response, "html.parser")
table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

try:
    location = table[0].text.split('Origin')[1].split('Genres')[0]

# save info in lists
    print('origin')

except:
    text = table[0].text

    # Step 1: Extract the part after 'Born'
    after_born = text.split("Born", 1)[1]

    text_age = re.search("aged", after_born)

    if text_age:
        # This means the artist is dead
        print('dead')
        location = re.split(r'(19\d{2})', after_born)[4].split('Died')[0].strip()
    else:
        try:
            text = re.split(r'(19\d{2})', after_born)[4].split(')')[1]

            if "Other\xa0names" in text:
                location = text.split('Other\xa0names')[0]
            else:
                if "Citizenship" in text:
                    location = text.split('Citizenship')[0]
                else:
                    if "Occupations" in text:
                        location = text.split('Occupations')[0]
                    else:
                        if "Genres" in text:
                            location = text.split('Genres')[0]
                            print(repr(location))
                        else:
                            location = np.nan
        except:  
            location = np.nan


In [535]:
text

'Seattle, Washington, U.S.GenresAlternative rockgrungeglam punkpunk rockhard rockheavy metalglam metalOccupationsMusiciansongwriterInstrumentsGuitarvocalsYears active'

In [536]:
location

'Seattle, Washington, U.S.GenresAlternative rockgrungeglam punkpunk rockhard rockheavy metalglam metal'

In [118]:
df.head()

,album_id,artist,album,rating,votes
0,37,Less Than Jake,Losing Streak,3.90,414
1,40,Sparta,Wiretap Scars,3.79,431
2,41,Muse,Absolution,3.99,4411
3,42,Muse,Showbiz,3.50,2181
4,45,Finch,What It Is to Burn,3.69,864


## Changing dataframes

### **``df_artists_origins``**

In [93]:
df_artists_origins = pd.read_csv('Datasets/df_artists_origins.csv')
df_artists_origins.shape

(3251, 2)

In [94]:
df_artists_origins.head()

,artist,origin
0,Sparta,"El Paso, Texas"
1,Muse,"Teignmouth, Devon, England"
2,Finch,"Temecula, California, Estados Unidos"
3,Transplants,"Los Angeles, California, United States[1]"
4,Rooney,"Los Angeles, California, U.S."


In [95]:
df_artists_origins[df_artists_origins['origin']=='United Kingdom']

,artist,origin


In [86]:
df_new_artists = df_artists_origins[df_artists_origins['origin']=='United Kingdom']

In [89]:
df_new_artists

,artist,origin
1059,Son of Dork,"London, England"
1192,Mojave 3,"London, England"
1326,Crippled Black Phoenix,"Bristol, England"
1597,This Mortal Coil,"Wandsworth, England"
2191,Arcadia,"London, England"
2300,Jade Warrior,"London, England"
2348,The Waterboys,"London, England"
2469,Blackmore's Night,"Mount Sinai, NY, United States"
2556,Atomic Rooster,"London, England"
2609,The Nefilim,"Lambeth, England"


In [88]:
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Son of Dork", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Mojave 3", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Crippled Black Phoenix", "Bristol, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="This Mortal Coil", "Wandsworth, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Arcadia", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Jade Warrior", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="The Waterboys", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Blackmore's Night", "Mount Sinai, NY, United States", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Atomic Rooster", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="The Nefilim", "Lambeth, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Black Spiders", "Sheffield, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Brontide", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Gilgamesh", "Hampstead, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Young Legionnaire", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="The Deviants", "Ladbroke Grove, London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Head of David", "Dudley, West Midlands, United Kingdom", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Quintessence", "London, England", df_new_artists["origin"])

In [53]:
# check if there's info of the artist origin in the column 'artist_profile'
import textwrap
artist_profile = df.loc[8016]['artist_profile']
splitted_string = textwrap.fill(artist_profile, width=120)
print(splitted_string)

Formed in 1969, they played a blend of jazz, progressive rock, Indian Music, and new age rock.   Members included:
Sambhu Babaji : Bass  Dave Codling : Guitar  Shiva Shankar Jones : Keyboards, Vocals  Jake Milton : Drums  Alan Mostert
: Guitar  Raja Ram : Flute, Piano, Vocals


In [52]:
# look for the albums of the artist in the original df to check it's the correct artist
df[df['artist']=="Quintessence".strip()].sort_values('votes', ascending=False).head()

,year,artist,title,rating,votes,album_length,tracks,styles,release_country,artist_profile
8016,1969,Quintessence,In Blissful Company,3.88,16,0.0,8,['Psychedelic Rock'],UK,"Formed in 1969, they played a blend of jazz, p..."


In [90]:
df_artists_origins = df_artists_origins[df_artists_origins['origin']!='United Kingdom']
df_artists_origins.shape

(3234, 2)

In [91]:
df_artists_origins.to_csv('Datasets/df_artists_origins.csv', index=False)

In [92]:
export_artists_origins_concat(df_new_artists)

df_artists_origins_concat exported to .csv
(3251, 2)


In [56]:
df_coordinates = get_coordinates_geopy(df_new_artists)
df_coordinates

1/9 - London, Greater London, England, United Kingdom
2/9 - Bristol, City of Bristol, West of England, England, United Kingdom
3/9 - Wandsworth, London Borough of Wandsworth, London, Greater London, England, SW18 1UJ, United Kingdom
4/9 - Mount Sinai, Miller Place, Town of Brookhaven, Suffolk County, New York, 11766, United States
5/9 - Lambeth, London Borough of Lambeth, London, Greater London, England, SE1 7JW, United Kingdom
6/9 - Sheffield, South Yorkshire, England, United Kingdom
7/9 - Hampstead, Greater London, England, NW3 1QG, United Kingdom
8/9 - Ladbroke Grove, Westway, Lancaster West Estate, North Kensington, Royal Borough of Kensington and Chelsea, London, Greater London, England, W10 5YG, United Kingdom
9/9 - Dudley, West Midlands, England, United Kingdom


,country,city,origin,origin_clean,latitude,longitude,address
0,United Kingdom,Bristol,"Bristol, England","Bristol, England",51.453802,-2.597298,"Bristol, City of Bristol, West of England, Eng..."
1,United Kingdom,Dudley,"Dudley, West Midlands, United Kingdom","Dudley, West Midlands, United Kingdom",52.511083,-2.081681,"Dudley, West Midlands, England, United Kingdom"
2,United Kingdom,Hampstead,"Hampstead, England","Hampstead, England",51.556530,-0.178301,"Hampstead, Greater London, England, NW3 1QG, U..."
3,United Kingdom,Ladbroke Grove,"Ladbroke Grove, London, England","Ladbroke Grove, London, England",51.517264,-0.211102,"Ladbroke Grove, Westway, Lancaster West Estate..."
4,United Kingdom,Lambeth,"Lambeth, England","Lambeth, England",51.495211,-0.116335,"Lambeth, London Borough of Lambeth, London, Gr..."
5,United Kingdom,London,"London, England","London, England",51.507446,-0.127765,"London, Greater London, England, United Kingdom"
6,United Kingdom,Sheffield,"Sheffield, England","Sheffield, England",53.380663,-1.470228,"Sheffield, South Yorkshire, England, United Ki..."
7,United Kingdom,Wandsworth,"Wandsworth, England","Wandsworth, England",51.457027,-0.193261,"Wandsworth, London Borough of Wandsworth, Lond..."
8,United States,Mount Sinai,"Mount Sinai, NY, United States","Mount Sinai, NY, United States",40.941066,-73.019455,"Mount Sinai, Miller Place, Town of Brookhaven,..."


### **``df_coordinates``**

In [57]:
df_coordinates_scraped = pd.read_csv('Datasets/df_coordinates.csv')
df_coordinates_scraped.shape

(1527, 7)

In [58]:
df_coordinates_scraped.head()

,country,city,origin,origin_clean,latitude,longitude,address
0,Australia,Adelaide,"Adelaide, South Australia, Australia","Adelaide, South Australia, Australia",-34.928181,138.599931,"Adelaide, Adelaide City Council, South Austral..."
1,Australia,Adelaide,"Adelaide, South Australia","Adelaide, South Australia",-34.928181,138.599931,"Adelaide, Adelaide City Council, South Austral..."
2,Australia,Blue Mountains,"Blue Mountains, NSW, Australia","Blue Mountains, NSW, Australia",-33.609741,150.405224,"Blue Mountains, New South Wales, Australia"
3,Australia,Brisbane,"Brisbane, Queensland, Australia","Brisbane, Queensland, Australia",-27.468968,153.023499,"City of Brisbane, Queensland, Australia"
4,Australia,Byron Bay,"Byron Bay, New South Wales, Australia","Byron Bay, New South Wales, Australia",-28.648333,153.617778,"Byron Bay, Byron Shire Council, New South Wale..."


In [59]:
df_coordinates_scraped[df_coordinates_scraped['city']=='United Kingdom']

,country,city,origin,origin_clean,latitude,longitude,address
702,United Kingdom,United Kingdom,United Kingdom,United Kingdom,54.702354,-3.276575,United Kingdom


In [60]:
df_coordinates_scraped.drop(702, axis=0, inplace=True)
df_coordinates_scraped[df_coordinates_scraped['city']=='United Kingdom']

,country,city,origin,origin_clean,latitude,longitude,address


In [62]:
df_coordinates_scraped.to_csv('Datasets/df_coordinates.csv', index=False)

In [63]:
df_coordinates.shape

(9, 7)

In [64]:
export_coordinates_concat(df_coordinates)

df_coordinates_scraped: (1526, 7)

Found 4 duplicates:
               city         country
324         Bristol  United Kingdom
515  Ladbroke Grove  United Kingdom
542          London  United Kingdom
650       Sheffield  United Kingdom

Resulting dataset: (1531, 7)
Merged artists with coordinates! Found 5 new locations
df_coordinates_concat exported to .csv


### **``df_artists_origins_coordinates_concat``**

In [74]:
df_artists_origins_coordinates_concat = merge_origins_coordinates(df_new_artists)
df_artists_origins_coordinates_concat.tail()

Exported to a .csv file


,artist,country,city,latitude,longitude,address
3246,Gilgamesh,United Kingdom,Hampstead,51.556530,-0.178301,"Hampstead, Greater London, England, NW3 1QG, U..."
3247,Young Legionnaire,United Kingdom,London,51.489334,-0.144055,"London, Greater London, England, United Kingdom"
3248,The Deviants,United Kingdom,Ladbroke Grove,51.517264,-0.211102,"Ladbroke Grove, Westway, Lancaster West Estate..."
3249,Head of David,United Kingdom,Dudley,52.511083,-2.081681,"Dudley, West Midlands, England, United Kingdom"
3250,Quintessence,United Kingdom,London,51.489334,-0.144055,"London, Greater London, England, United Kingdom"


## Changing values

In [72]:
df_masters_blended[df_masters_blended['artist'].str.contains('Nucleus')]

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile
1004,3821235,1537522,13529787,US,Nucleus (US),Entity,2019,38.42,8,['Album'],['Rock'],['Death Metal'],"Death Metal band from Chicago, Illinois, USA. ..."
1592,3821235,1094310,8362817,US,Nucleus (US),Sentient,2016,37.92,9,['Album'],['Rock'],['Death Metal'],"Death Metal band from Chicago, Illinois, USA. ..."
5257,184256,175620,279855,UK,Nucleus (UK),We'll Talk About It Later,1971,0.00,7,"['LP', 'Album']","['Jazz', 'Rock']","['Fusion', 'Jazz-Funk', 'Jazz-Rock', 'Prog Rock']","Pioneering jazz-rock, progressive, psychedelic..."
10195,184256,23574,465143,UK,Nucleus (UK),Elastic Rock,1970,0.00,13,"['LP', 'Album']","['Jazz', 'Rock']","['Jazz-Rock', 'Fusion', 'Prog Rock']","Pioneering jazz-rock, progressive, psychedelic..."


In [75]:
df_masters_blended[df_masters_blended['title'].str.contains('Alleycat')]

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile


In [77]:
df_ratings_20[df_ratings_20['artist'].str.contains('Nucleus')]

,album_id,artist,title,rating,votes
29004,86398,Nucleus (UK),Elastic Rock,3.55,20
29120,87140,Nucleus (UK),We'll Talk About It Later,3.79,21
39636,216509,Nucleus (US),Sentient,3.29,29
45524,333839,Nucleus (US),Entity,3.68,40
46380,352917,Nucleus (UK),Alleycat,3.52,20


In [76]:
df_ratings_20.loc[46380, 'artist'] = 'Nucleus (UK)'
df_ratings_20.loc[29004, 'artist'] = 'Nucleus (UK)'
df_ratings_20.loc[29120, 'artist'] = 'Nucleus (UK)'
df_ratings_20.loc[39636, 'artist'] = 'Nucleus (US)'
df_ratings_20.loc[45524, 'artist'] = 'Nucleus (US)'

In [869]:
df.drop([7660, 8037], axis=0, inplace=True)
df.shape

(12059, 10)

In [122]:
df['artist'] = np.where(df['artist']=='pg.99 / Majority Rule', 'Majority Rule', df['artist'])
df[df['artist']=='pg.99 / Majority Rule']

,album_id,artist,album,rating,votes


In [78]:
df_ratings_20.to_csv('Datasets/df_ratings_20.csv', index=False)

In [581]:
df.shape

(12059, 10)

### **Testing code for strange cases**

In [ ]:
df = pd.read_csv('Datasets/df_ratings.csv')
artists = df['artist'].unique()

artists_list = []
origin_list = []
count=0

for index in artists[140:150]:

    artists_list.append(index)
    name_changed = index.replace(' ', '_')
    name_changed_band = name_changed + ('_(band)')

    try:
        url = f"https://en.wikipedia.org/wiki/{name_changed_band}"
        response = requests.get(url).content
        soup = BeautifulSoup(response, "html.parser")

        origin = soup.select('table tr th', class_='infobox-label')

        if len(origin) > 0:
            try:
                if origin[2].text == 'Origin':
                    location = soup.select('table tr td', class_='infobox-data')[1].text
                elif origin[3].text == 'Origin':
                    location = soup.select('table tr td', class_='infobox-data')[2].text
                # else:
                    
                if 'multiple issues' in location:
                    location = soup.select('table tr td', class_='infobox-data')[7].text        
                    print(f'{count} - {name_changed_band}: multiple issues - {location}')
                    origin_list.append(location)
                elif 'additional citations' in location:
                    location = soup.select('table tr td', class_='infobox-data')[3].text        
                    print(f'{count} - {name_changed_band}: additional citations - {location}')
                    origin_list.append(location)

                else:
                    print(f'{count} - {name_changed_band}: {location}')
                    origin_list.append(location)
            except:
                print(f'{count} - {name_changed_band}: {location}')
                origin_list.append(location)      
        else:
            try:
                url = f"https://en.wikipedia.org/wiki/{name_changed}"
                response = requests.get(url).content
                soup = BeautifulSoup(response, "html.parser")

                origin = soup.select('table tr th', class_='infobox-label')

                if len(origin) > 0:
                    if origin[2].text == 'Origin':
                        location = soup.select('table tr td', class_='infobox-data')[1].text

                        if 'multiple issues' in location:
                            location = soup.select('table tr td', class_='infobox-data')[7].text        
                            print(f'{count} - {name_changed_band}: multiple issues - {location}')
                            origin_list.append(location)
                        elif 'additional citations' in location:
                            location = soup.select('table tr td', class_='infobox-data')[3].text        
                            print(f'{count} - {name_changed_band}: additional citations - {location}')
                            origin_list.append(location)
                        else:
                            print(f'{count} - {name_changed_band}: {location}')
                            origin_list.append(location)

                    elif origin[3].text == 'Origin':
                        location = soup.select('table tr td', class_='infobox-data')[2].text
                        print(f'{count} - {name_changed_band}: {location}')
                        origin_list.append(location) 

                    else:
                        print(f'{count} - {index}: no location found')
                        origin_list.append(np.nan)  
                else:
                    print(f'{count} - {index}: short length')
                    origin_list.append(np.nan)
            except:
                print(f'{count} - {index}: error')
                origin_list.append(np.nan)
    except:
        print(f'{count} - {index}: error')
        origin_list.append(np.nan)

    if len(artists_list) != len(origin_list):
        print('different lengths')
        break

    count+=1

0 - Funeral_for_a_Friend_(band): Bridgend, Wales
1 - Millencolin_(band): multiple issues - Örebro, Sweden
2 - The_Flaming_Lips_(band): Oklahoma City, Oklahoma, U.S.
3 - Feeder_(band): Feeder in 2008
4 - Descendents_(band): Manhattan Beach, California, U.S.
5 - PJ Harvey: no location found
6 - Godsmack_(band): Lawrence, Massachusetts U.S.
7 - Blind_Faith_(band): Ripley, Surrey, England
8 - Van_Halen_(band): Pasadena, California, U.S.
9 - Damageplan_(band): Dallas, Texas, U.S.


## Defining the functions

In [15]:
def get_origins_wikipedia(df, start_index, final_index):
    df = pd.read_csv('Datasets/df_rock_ratings_20.csv')
    artists = df['artist'].unique()

    try:
    # import the DataFrame with the locations whose coordinates I already have
        df_coordinates_scraped = pd.read_csv('Datasets/df_coordinates.csv')
        print('Bingo! df_coordinates.csv found \n')
    except: 
        print('df_coordinates.csv not found \n')

    artists_list = []
    origin_list = []
    count=0
    scraped=0

    for index in artists_us_to_do[start_index:final_index]:

        name_changed = index.replace(' ', '_')
        name_changed_band = name_changed + ('_(band)')

        try:
            url = f"https://en.wikipedia.org/wiki/{name_changed_band}"
            response = requests.get(url).content
            soup = BeautifulSoup(response, "html.parser")

            table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

            location = table[0].text.split('Origin')[1].split('Genres')[0]
            city = location.split(', ')[0]
            count+=1
            
        # save info in lists
            artists_list.append(index)  
            origin_list.append(location)
            scraped+=1
            print(f'{scraped}/{count} - {name_changed_band}: {location}')

        except:
            try:
                url = f"https://en.wikipedia.org/wiki/{name_changed}"
                response = requests.get(url).content
                soup = BeautifulSoup(response, "html.parser")
                table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

                try:
                    location = table[0].text.split('Origin')[1].split('Genres')[0]
                    city = location.split(', ')[0]
                    count+=1 
    
                # save info in lists
                    artists_list.append(index)  
                    origin_list.append(location)
                    scraped+=1
                    print(f'{scraped}/{count} - {name_changed}: {location}')

                except:
                    location = table[0].text.split(')')[2].split('Genres')[0]
                    city = location.split(', ')[0]
                    count+=1

                # save info in lists
                    artists_list.append(index)  
                    origin_list.append(location)
                    scraped+=1
                    print(f'{scraped}/{count} - {name_changed} (individual): {location}')

            except:
                try:
                    url = f"https://es.wikipedia.org/wiki/{name_changed}"
                    response = requests.get(url).content
                    soup = BeautifulSoup(response, "html.parser")

                    table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')
                    location = table[0].text.split('Origen\n')[1].split(' Información')[0]
                    city = location.split(', ')[0]
                    count+=1    
    
                # save info in lists
                    artists_list.append(index)  
                    origin_list.append(location)
                    scraped+=1
                    print(f'{scraped}/{count} - {name_changed} (español): {location}')

                except:
                    count+=1
                    print(f'{scraped}/{count} - {index}: error')
                    artists_list.append(index) 
                    origin_list.append(np.nan)

        if len(artists_list) != len(origin_list):
            print('different lengths')
            break

    df_artists_origins = pd.DataFrame({'artist': artists_list
                             , 'origin': origin_list})
    
    return df_artists_origins

In [102]:
def get_new_artists(df_artists_origins):

# import the df with the artists' origins already scraped
    df_artists_origins_scraped = pd.read_csv('Datasets/df_artists_origins.csv')

    if df_artists_origins['origin'].isna().sum() == 0:        
        print("No null values, but let's take a look just in case there are weird locations")

    else: 
    # take a look at the df with the new artists and make sure there are non null values in origin (when it couldn't find it in Wikipedia)
        print(f'{round(df_artists_origins['origin'].isna().sum() / df_artists_origins.shape[0]*100, 2)} % of nulls')
    
# subset of the new artists I just got, wether there are null values or not
    df_new_artists = df_artists_origins[~df_artists_origins['artist'].isin(df_artists_origins_scraped['artist'].values)]

    print("Here is the dataframe with the new artists, without duplicates")
    return df_new_artists   # so I can take a look at it and then continue

In [ ]:
def export_artists_origins_concat(df_new_artists):

# import the df with the artists' origins already scraped
    df_artists_origins_scraped = pd.read_csv('Datasets/df_artists_origins.csv')

# concat with the df I just got
    df_artists_origins_concat = pd.concat([df_artists_origins_scraped, df_new_artists])
    df_artists_origins_concat.drop_duplicates(inplace=True)     # just in case
    df_artists_origins_concat.reset_index(drop=True, inplace=True)

# export all the artists and their origins to a .csv file (the ones I got plus the new artists)
    df_artists_origins_concat.to_csv('Datasets/df_artists_origins.csv', index=False)
    print('df_artists_origins_concat exported to .csv')
    print(df_artists_origins_concat.shape)

In [175]:
def get_coordinates_geopy(df_new_artists):
    
# replace special characters for spaces
    df_new_artists['origin_clean'] = df_new_artists['origin'].str.replace('.', '')
    df_new_artists['origin_clean'] = df_new_artists['origin_clean'].str.replace(r'\[\d+\]', '', regex=True)

# run the function that gets the coordinates from the origins from Geopy
    geolocator = Nominatim(user_agent="music_analysis", timeout=10)

# if they are 'dirty' origins that after the cleaning, they result in the same 'origin_clean'
    df_unique = df_new_artists[['origin', 'origin_clean']].drop_duplicates() 
    unique_origins = df_unique['origin'].values
    unique_origins_clean = df_unique['origin_clean'].values

    country_list = []
    city_list = []
    latitude_list = []
    longitude_list = []
    address_list = []
    lists = [country_list, city_list, latitude_list, longitude_list, address_list]
    count = 0

    for origin in unique_origins_clean:
        count+=1
        time.sleep(1)
        location = geolocator.geocode(origin)

        print(f'{count}/{len(unique_origins_clean)} - {location.address}')  

    # save the info in lists
        country_list.append(location.address.split(', ')[-1])
        city_list.append(origin.split(', ')[0])
        latitude_list.append(location.latitude)
        longitude_list.append(location.longitude)
        address_list.append(location.address)

        # # Check lengths
        # print(f"{count}/{len(unique_origins_clean)} - {origin}")
        # print(f"Current list lengths -> country: {len(country_list)}, city: {len(city_list)}, "
        #     f"lat: {len(latitude_list)}, lon: {len(longitude_list)}, address: {len(address_list)}")

    df_coordinates = pd.DataFrame({'country': country_list
                                , 'city': city_list
                                , 'origin': unique_origins
                                , 'origin_clean': unique_origins_clean
                                , 'latitude': latitude_list
                                , 'longitude': longitude_list
                                , 'address': address_list})
    df_coordinates.sort_values(['country', 'city'], inplace=True) # sort by country and city
    df_coordinates.reset_index(drop=True, inplace=True)

    return df_coordinates

In [ ]:
def export_coordinates_concat(df_coordinates):

# import the last df that contains the coordinates of the unique origins
    df_coordinates_scraped = pd.read_csv('Datasets/df_coordinates.csv')
    print(f"df_coordinates_scraped: {df_coordinates_scraped.shape}\n")

# concat with the df of the coordinates I just got
    df_coordinates_concat = pd.concat([df_coordinates_scraped, df_coordinates])
    df_coordinates_concat.sort_values(['country', 'city'], inplace=True) # sort by country and city
    df_coordinates_concat.reset_index(drop=True, inplace=True)

# look for duplicates in the origin, between the locations I had already gotten and the new ones
    check_duplicates_origins(df_coordinates_concat)
    new_origins = df_coordinates_concat.shape[0] - df_coordinates_scraped.shape[0]
    print(f"Merged artists with coordinates! Found {new_origins} new locations")

# save it in a csv file (the coordinates I had plus the ones from the new artists I just got)
    df_coordinates_concat.to_csv('Datasets/df_coordinates.csv', index=False)
    print('df_coordinates_concat exported to .csv')

In [140]:
def merge_origins_coordinates(df_new_artists):

# import the last df that contains the coordinates of the unique origins
    df_coordinates_concat = pd.read_csv('Datasets/df_coordinates.csv')

# merge with the previous df with the artists
    df_artists_origins_coordinates = pd.merge(df_new_artists, df_coordinates_concat, on=['origin'])
    df_artists_origins_coordinates.drop(columns=['origin', 'origin_clean_x', 'origin_clean_y'], inplace=True)

# import the df that contains info of the artists and the coordinates of their origins
    df_artists_origins_coordinates_scraped = pd.read_csv('Datasets/df_artists_origins_coordinates.csv')

# concat to get the df with all the artists, origins and their coordinates
    df_artists_origins_coordinates_concat = pd.concat([df_artists_origins_coordinates_scraped, df_artists_origins_coordinates])
    df_artists_origins_coordinates_concat.reset_index(drop=True, inplace=True)

# save it in a csv file
    df_artists_origins_coordinates_concat.to_csv('Datasets/df_artists_origins_coordinates.csv', index=False)
    print("Exported to a .csv file")

    return df_artists_origins_coordinates_concat

## Merge dataframes and look for the ``new_artists``

In [11]:
artists_blend = df_masters_blended['artist'].unique()
len(artists_blend)

4527

In [12]:
df_artists_origins = pd.read_csv('Datasets/df_artists_origins.csv')
artists = df_artists_origins['artist'].unique()
artists_usa = []

for artist in artists_blend:
    if artist not in df_artists_origins['artist'].values:
        artists_usa.append(artist)

len(artists_usa)

1555

In [13]:
artists_usa

['As Living Arrows',
 'Hidden Mothers',
 'Tiny Moving Parts',
 'Poppy',
 'State Champs',
 'Oso Oso',
 'Better Lovers',
 'Lowen',
 'Halsey',
 'Amyl and the Sniffers',
 'Delta Sleep',
 'High Vis',
 'Cemetery Skyline',
 'Goat',
 'Chat Pile',
 'Drug Church',
 'Origami Angel',
 'Heriot',
 'Nightwish',
 'Foxing',
 'Alora Crucible',
 'Wage War',
 'TURQUOISEDEATH',
 'Dawn Treader',
 'Boston Manor',
 'MJ Lenderman',
 'Fat Dog',
 'Kingcrow',
 'Leprous',
 'Wunderhorse',
 'thrown',
 'Horse Jumper of Love',
 'Within the Ruins',
 'Magdalena Bay',
 'Fontaines D.C.',
 'beabadoobee',
 'State Faults',
 'Graphic Nature',
 'The Home Team',
 'Speed',
 'Remi Wolf',
 'Clairo',
 'Crippling Alcoholism',
 'Cigarettes After Sex',
 'Abriction',
 'Pijn',
 'Outlander',
 'Imagine Dragons',
 'The Dangerous Summer',
 'Pond',
 'Hyperdontia',
 'Vredehammer',
 'Pedro the Lion',
 'Weston Super Maim',
 'Stand Still',
 'Mortal Wound',
 'Eye of Solitude',
 'Beth Gibbons',
 'Contention',
 'Knocked Loose',
 'Mdou Moctar',
 'Th

In [110]:
# I have to import this df for the function to properly work
df = pd.read_csv('Datasets/df_blend_ratings.csv')
artists = df['artist'].unique()
len(artists)

4249

In [11]:
artists[-1]

'As Living Arrows'

In [12]:
df_artists_origins.index[-1]

3423

In [13]:
df_artists_origins.tail()

,artist,origin
3419,Cinderella,"Philadelphia, Pennsylvania, U.S."
3420,Death From Above 1979,"Toronto, Ontario, Canada"
3421,Chicago,"Chicago, Illinois, United States"
3422,The Nation of Ulysses,"Washington, D.C."
3423,Strung Out,"Simi Valley, California, U.S."


In [164]:
np.where(artists=='July Talk')

(array([2772], dtype=int64),)

In [163]:
artists[2772]

'July Talk'

# **Code to execute the functions from ``geopy_functions.py``**

In [2]:
artists_to_remove = ['Cemetery Skyline', 'Goat', 'Kingcrow', 'Speed', 'Hyperdontia', 'Vredehammer', 'Weston Super Maim',
                'Mdou Moctar', 'AVRALIZE', 'Engulfed', 'Coffin Storm', 'samlrc', 'Little Kid', 'Termina', 'Rorcal',
                'Reverence To Paroxysm', 'Voyager', 'Blindfolded and Led to the Woods', 'Mork', 'Yeule', 'Pond',
                'Empire State Bastard', 'Blood Command', 'Avatar', 'Brutus', 'Faceless Burial', 'Within Destruction',
                'Sedimentum', 'Ataraxy', 'Corpsessed', 'Epitaphe', 'Aldous Harding', 'Messa', 'Ghost', 'Ecchymosis',
                'Wormrot', 'Vertebra Atlantis', 'Papangu', 'Sermon of Flames', 'Springtime', 'Monolord', 'Sulphurous',
                'LLNN', 'Trna', 'Slaughter To Prevail', 'sonhos tomam conta', 'Galvanizer', 'Ophidian I', 'Diabolizer',
                'Morbific', 'Defacement', 'Divide And Dissolve', 'Soen', 'Scorpions', 'Accept', 'Respire', 'Undergang',
                'Bearings', 'Scalp', 'Miasmatic Necrosis', 'Teenage Mutant Ninja Turtles', 'Gorephilia', 'Vous Autres',
                'Carnation', 'Pharmacist', 'Paara', "Justice For The Damned", 'VVilderness', 'Molested Divinity', 'Ellis',
                'Kontinuum', 'Monsters', 'Giver', 'Lowrider', 'Vengeful Spectre', 'Vomit the Soul', 'Sadisme', 'Alarmist',
                'Klone', 'Nocturnal Departure', 'King Gizzard and The Lizard Wizard', 'Make Them Suffer', 'The Chats',
                'Patrick Watson', 'Shirokuma', 'Forests', 'Town Portal', 'Ceremony Of Silence', 'CHAI', 'Baalsebub',
                'Minors', 'Mono', 'Tallies', 'Normandie', 'Mouse On The Keys', 'Burial Invocation', 'Orville Peck',
                'Lunatic Soul', 'Alex Lahey', 'Hozier', 'Mystifier', 'Hands Like Houses', 'Ruins', 'Autokrator',
                'Legend of the Seagullmen', 'Death Toll 80k', 'IDYLLS', 'Spaceslug', 'i hate sex', 'Band-Maid',
                'With the Dead', 'Hungry Ghosts', 'Middle Kids', 'Gleb Kolyadin', "Leaves' Eyes", "Phrenelith",
                "David Brent", "Art Of Dying", "Minor Victories", "Pantha Du Prince", "Schammasch", 'LUH',
                'Violet Cold', 'EL VY', 'Shining', 'Hills', "Mutiny On The Bounty", 'Lucifer', 'FFS', 'Ranger',
                'Trepalium', 'A Textbook Tragedy', 'This Is A Standoff', 'FACT', 'Swallowed', 'Disembarked',
                'Wildbirds and Peacedrums', 'Archivist', 'Timber Timbre', 'Newsboys', 'Dope Lemon', 'Vagabon',
                'RY X', 'Moonlit Sailor', 'The Drones', 'Sarah Neufeld', 'Say Lou Lou', 'Cruciamentum', 'Lay Down Rotten',
                'Dreamshade', 'Sickening Gore', 'Circles', "Avey Tare's Slasher Flicks", 'Forest Silence',
                "One Eyed God Prophecy", 'Coffins', 'Osamu Kitajima', 'Living With Lions', 'Ansur', 'Parades',
                "Intestine Baalism", 'Comity', 'No Omega', 'Wolverine', 'Disavowed', 'Angel Dust', "!T.O.O.H.!",
                'Hypnosia', 'Hexenhaus', 'Paradox', 'Deathrow', 'Excruciate', 'FareWell Poetry', 'Sights and Sounds',
                'Supersister', "Birds Of Tokyo", 'Ark', "The Flower Kings", 'Beardfish', 'Graveworm', 'Acid',
                'Ladyhawke', 'Geddy Lee', 'Yngwie Malmsteen', "World's End Girlfriend", 'Totem Skin', 'Lewis',
                'I Hate Sally', "The Band", 'Lisa Hannigan', 'Lethal', 'Bubu', 'Van She', 'Mooncake', 'The Haunted',
                "Orphaned Land", 'Madder Mortem', 'Kataxu', 'Gilberto Gil', 'Vendetta', 'Kvist', 'Acrostichon', 'Pain',
                'Obliteration', 'Flames of Hell', 'Wombbath', 'Stone', 'Disgrace', 'Fionn Regan', 'Disastrous Murmur',
                'Urfaust', 'Sleepingdog', 'Island', 'Bethlehem', 'Subterranean Masquerade', 'After Dinner', 
                'Black Boned Angel', 'FM', 'Embrace', 'Solefald', 'Maneige', 'Amberian Dawn', 'OOIOO', 'Anekdoten',
                "Aphrodite's Child", 'Hollenthon', 'Lykke Li', 'Lenka', 'Sarah McLachlan', 'Owen Pallett',
                'Devin Townsend Project', 'Missy Higgins', 'The Devin Townsend Band', 'Selda', 'Massacra', "Rory Gallagher",
                'Taste', 'Celestial Season', 'Ida Maria', 'Dark Tranquillity', 'Cadaver', 'Pele', 'Exuma',
                'Great Lake Swimmers', 'Dawn', 'The Bats', 'Yoko Ono', 'Illogicist', 'The Saints', 'Final Fantasy',
                'Pendulum', 'Lunar Aurora', 'Bee Gees', 'Stars', "David Sylvian and Robert Fripp", 'Afflicted', 'Lengsel',
                'Extol', 'MDFMK', 'Univers Zero', 'Mortuary Drape', 'Zyklon', 'Winds', 'Zyklon-B', 'The Sins of Thy Beloved',
                'Lords of Acid', 'Devin Townsend', 'Diablo Swing Orchestra', 'Arcturus', 'Cornelius', 'Manu Chao',
                'Bryan Adams', 'Peaches', 'Doro', 'Kingdom Come', 'Pekka Pohjola', 'Shakira', 'Massacre', 'Subhumans',
                'Set Fire to Flames', 'Gorgoroth', 'Gandalf', 'Klaus Schulze', 'The Ecstasy of Saint Theresa',
                "Lou Reed and John Cale", 'Brian Eno and David Byrne', 'Bob Dylan and The Band', 'Era', 'Devil Doll',
                'Cauterize', 'Roadrunner United', 'Circus Maximus', 'Crowpath', 'Raunchy', 'Tad Morose', 'Kenna',
                'Head Control System', 'Torchbearer', 'Rosesdead', 'Angtoria', 'Nightrage', 'Necros Christos', 'Hypnos 69',
                'Wold', 'Andromeda', 'Chad VanGaalen', 'Melechesh', 'Spheric Universe Experience', 'Anubis Gate',
                'The Project Hate MCMXCIX', 'Myrath', 'Savage Circus', 'Hartfield', 'Evereve', 'Daturah', 'Ad Infinitum',
                'Tash Sultana', 'Tarja Turunen', 'Ram-Zet', 'Sweven', 'Arcane', 'Sons of Apollo', 'Celesty', 'Brainstorm',
                'Unleash The Archers', 'Prostitute Disfigurement', 'The Psyke Project', 'Dornenreich', 'Watain', 'Funeral',
                'Cultes Des Ghoules', 'Revolting', 'Igorrr', 'Symfonia', 'Darkestrah', 'Sarah Blasko', 'Fractal Universe',
                'The End', 'Apotheosis', 'Drautran', 'Monolithe', 'CrazyEightyEight', 'Black Hole', 'Polaris', 'Nug',
                'Klabautamann', 'James LaBrie', 'Vance Joy', 'Helena Deland', 'Authorize', 'Blazon Stone', 'Rapture',
                'Worship', 'Conqueror', 'Swan Lake', 'Yyrkoon', 'Inquisition', 'Kerli', 'Keldian', 'Wake', 'Megiddo',
                'Juanes', 'William Hung', 'Votum', 'Atramentus', 'Abysmal Torment', 'Paul Dempsey', 'Cytotoxin',
                'Vulture Industries', 'Entrails', 'Jorn', 'Conception', 'Centaurus-A', 'Gezan', 'Blood Tsunami',
                'Machinemade God', 'Proscription', 'Ragnarok', 'Zeromancer', 'Satariel', 'Circle of Ouroborus', 'Emeth',
                'Before The Dawn', 'Near Death Condition', 'Ignivomous', 'MEANS', 'Svart Crown', 'Alan Sorrenti', 'Chthonic',
                'Blues Pills', 'Majestica', 'Bedsore', 'Ravencult', 'William Shatner', 'Cheval De Frise', 'Emptiness',
                'WHOURKR', 'Jet Black Stare', 'Despondency', 'October Falls', 'Istapp', 'Slumber', 'Night in Gales',
                'Ov HELL', 'Aficionado', 'Old Silver Key', 'Junior Battles', 'Adversarial', 'Shugo Tokumaru', 'North',
                'Fuck on the Beach', 'Anoice', 'Viscera', 'Two Tongues', 'The Last Felony', 'Revenge', 'The Secret',
                'Cosmic Putrefaction', 'Miseration', 'Azusa', 'Stalaggh', 'Cerebral Effusion', 'Nekromantheon', 'Blasphemer',
                'Pyaemia', 'Inveracity', 'Orchidectomy', 'Kraanium', 'Human Mincer', 'Deformity', 'Kaospilot',
                'Abominable Putridity', 'Iskra', 'Das Oath', 'Torsofuck', 'Internal Suffering', 'Nerlich', 'Blasphemophagher',
                'Head Wound City', 'Inanna', 'Disperse', 'Dishammer', 'Damaar', 'Extortion', 'Starring Janet Leigh',
                'The Arcane Order', "D'espairsRay", 'Demians', "Dave Matthews"]

In [3]:
df_final = pd.read_csv('Datasets/df_final.csv')
unique_artists = df_final[df_final['year']<2011]['artist'].unique()

df_artists_origins = pd.read_csv('Datasets/df_artists_origins.csv')
artists = df_artists_origins['artist'].unique()
artists_to_do = []

for artist in unique_artists:
    if artist not in artists and artist not in artists_to_remove:
        artists_to_do.append(artist)

len(artists_to_do)

1

In [4]:
# show the first next 5 artists I'm going to scrape
artists_to_do[:5]

['Grateful Dead']

In [5]:
# create the df with the origins scraped from Wikipedia

df = pd.read_csv('Datasets/df_final.csv')
start_index = 0
final_index = start_index+1

df_artists_origins = get_origins_wikipedia(df, start_index, final_index)

Bingo! df_coordinates.csv found 

1/1 - Grateful_Dead: Palo Alto, California, U.S.


In [6]:
df_artists_origins[df_artists_origins['artist']=='Grateful Dead']

,artist,origin
0,Grateful Dead,"Palo Alto, California, U.S."


In [7]:
# create a subset of the new artists I just got, and tell me if there are nulls
df_new_artists = get_new_artists(df_artists_origins)

# show the first new artists, if they were duplicates they have been dropped
df_new_artists.head()

No null values, but let's take a look just in case there are weird locations


,artist,origin
0,Grateful Dead,"Palo Alto, California, U.S."


### If there are null or weird values

In [14]:
# create a df with where origin is a null value
nulls = df_new_artists[df_new_artists['origin'].isna()]
nulls.shape

(0, 2)

**``np.where`` to replace the values for the real origins**

In [159]:
# so I can print the np.where and I save time
for artist in nulls['artist'].values:
    print(f'df_new_artists["origin"] = np.where(df_new_artists["artist"]=="{artist}", "England", df_new_artists["origin"])')

df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Black Math Horseman", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Broken Water", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Cire", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="The Arcane Order", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="The Reptilian", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="The Riot Before", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Brother Von Doom", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Orchid", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Addaura", "E

**Looking in the internet for the real origins of these artists**

In [160]:
nulls.head(10) # so it's faster to copy the names of the artists to look for their origins on the internet

,artist,origin
192,Black Math Horseman,NaN
194,Broken Water,NaN
197,Cire,NaN
200,The Arcane Order,NaN
201,The Reptilian,NaN
206,The Riot Before,NaN
208,Brother Von Doom,NaN
209,Orchid,NaN
210,Addaura,NaN
215,Until Your Heart Stops,NaN


In [184]:
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Black Math Horseman", "Los Angeles", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Broken Water", "Olympia, Washington", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Cire", "New Orleans, LA, United States", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="The Reptilian", "Kalamazoo, Michigan", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="The Riot Before", "Richmond, VA, United States", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Brother Von Doom", "Dayton, OH, United States", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Orchid", "Amherst, Massachusetts", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Addaura", "Seattle, Washington", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Until Your Heart Stops", "Oakland, CA, United States", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Only Crime", "Fort Collins, CO, United States", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="While Heaven Wept", "Dale City, VA, United States", df_new_artists["origin"])

In [178]:
# look for the albums of the artist in the original df to check it's the correct artist
df[df['artist']=="Only Crime	".strip()].sort_values('year').head()

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile
5619,305973,386259,3227765,US,Only Crime,Virulence,2007,0.0,12,['Album'],['Rock'],['Punk'],Melodic hardcore band formed by [a=Good Riddan...


In [179]:
# check if there's info of the artist origin in the column 'artist_profile'
import textwrap
artist_profile = df.loc[5619]['artist_profile']

splitted_string = textwrap.fill(artist_profile, width=120)
print(splitted_string)

Melodic hardcore band formed by [a=Good Riddance] singer [a=Russ Rankin] and [a=Bane (2)] guitarist [a=Aaron Dalbec] in
2003.


#### Changing the original dataframes in case needed (ex: two bands with the same name)

In [73]:
df.loc[11090, 'artist'] = 'Embrace (US)'
df.loc[6140, 'artist'] = 'Embrace (UK)'
df.loc[6141, 'artist'] = 'Embrace (UK)'

In [74]:
df.to_csv('Datasets/df_masters_blended.csv', index=False)

In [75]:
# look for the albums of the artist in the original df_ratings_20 to check it's the correct artist
df_ratings_20[df_ratings_20['artist']=="Embrace".strip()]

,album_id,artist,title,rating,votes
4485,7197,Embrace,The Good Will Out,3.56,24
4486,7198,Embrace,Out Of Nothing,3.39,27
8649,14575,Embrace,Embrace,3.99,309


In [76]:
df_ratings_20.loc[8649, 'artist'] = 'Embrace (US)'
df_ratings_20.loc[4486, 'artist'] = 'Embrace (UK)'
df_ratings_20.loc[4485, 'artist'] = 'Embrace (UK)'

In [77]:
df_ratings_20.to_csv('Datasets/df_ratings_20.csv', index=False)

In [214]:
df_new_artists['origin'].isna().sum()

0

#### **Drop artists that are not from the UK or the US** (or supergroups that don't have a clear origin)

In [ ]:
artists_to_remove = []

for artist in artists_to_remove:
    try:
        artists_usa.remove(artist)
        print(f'{artist} removed')
    except:
        pass

In [193]:
# I can drop single rows or I can just create a subset with the artists that I want to keep, the ones that are not in the list of artists to remove
print(f"Rows before dropping: {df_new_artists.shape[0]}")
df_new_artists = df_new_artists[~df_new_artists['artist'].isin(artists_to_remove)]
print(f"Rows after dropping: {df_new_artists.shape[0]}")

Rows before dropping: 207
Rows after dropping: 206


#### **Check short and long origins, probably wrong**

In [187]:
# print abnormally short origins and visually check if they are correct
for index, row in df_new_artists.iterrows():
    if len(row['origin']) < 10:
        print(index, row['origin'])

56 Fargo, ND
178 Boston


In [181]:
start_index = 207

end_index = start_index+5
df_new_artists.loc[start_index:end_index]

,artist,origin
207,While Heaven Wept,Virginia
208,Brother Von Doom,"Dayton, OH, United States"
209,Orchid,"Amherst, Massachusetts"
210,Addaura,"Seattle, Washington"
211,See You Next Tuesday,"Bay City, Michigan, United States"
212,Saints Never Surrender,"Fort Wayne, Indiana"


In [204]:
# look for the albums of the artist in the original df to check it's the correct artist
df[df['artist']=="Big Kids	".strip()].sort_values('year').head()

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile
6396,152830,451305,2572483,US,Big Kids,Hoop Dreams,2010,0.0,8,['LP'],['Rock'],"['Rock & Roll', 'Punk']","Punk band from Oakland, CA, USA."


In [201]:
# check if there's info of the artist origin in the column 'artist_profile'
import textwrap
artist_profile = df.loc[5587]['artist_profile']

splitted_string = textwrap.fill(artist_profile, width=120)
print(splitted_string)

American melodic hardcore punk band from Oakland, CA.


**Genres**

In [188]:
genres = df_new_artists[df_new_artists['origin'].str.contains('Genres')]
genres.shape

(7, 2)

**``np.where`` to replace the values for the real origins**

In [ ]:
for artist in genres['artist'].values:
    print(f'df_new_artists["origin"] = np.where(df_new_artists["artist"]=="{artist}", "UK", df_new_artists["origin"])')

In [194]:
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Derek Sherinian", "Laguna Beach, California, U.S.", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Jeff Loomis", "Appleton, Wisconsin, U.S.", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Johnny Hobo and the Freight Trains", "Brattleboro, Vermont, United States", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Bumblefoot", "Brooklyn, New York, U.S.", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Ill Bill", "New York City, U.S.", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Karl Sanders", "San Francisco, California, U.S.", df_new_artists["origin"])

In [195]:
# print the df so I can copy the origin from the column (before 'Genres')
df_new_artists[df_new_artists['origin'].str.contains('Genres')]

,artist,origin


In [206]:
# I create a new column to calculate the lenght of the origin, if it's long it probably didn't scrap correctly Wikipedia
df_new_artists["origin_length"] = df_new_artists["origin"].str.len()
long_strings = df_new_artists[df_new_artists["origin_length"]>40] # create a df based on these long origins
long_strings.shape # to see how many artists I have to take care of

(3, 3)

In [207]:
long_strings # display the df so I can copy the parts I am interested in

,artist,origin,origin_length
15,Latterman,"Huntington Station, New York, United States",43
147,Make Do and Mend,"West Hartford, Connecticut, United States",41
186,Baths,"Culver City, Los Angeles, California, United S...",51


In [ ]:
for artist in long_strings['artist'].values:
    print(f'df_new_artists["origin"] = np.where(df_new_artists["artist"]=="{artist}", "UK", df_new_artists["origin"])')

In [ ]:
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Heartsounds", "Oakland, CA", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Burn Halo", "Costa Mesa", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Gayngs", "Minneapolis, MN, United States", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Sound Tribe Sector 9", "Snellville, GA, United States", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Big Kids", "Oakland, CA", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="SQRM", "Springfield, Massachusetts", df_new_artists["origin"])

In [209]:
# I drop the column I just created of 'origin_length'
df_new_artists = df_new_artists[['artist', 'origin']]

In [210]:
# bands from United Kingdom, wrong origin, poor level of detail
df_new_artists[df_new_artists['origin']==('United Kingdom')]

,artist,origin


### **Changing easy values: individuals that didn't get the right origin in Wikipedia**

In [119]:
# individuals that didn't get the right origin in Wikipedia
df_new_artists[df_new_artists['origin'].str.contains(' and ')]

,artist,origin


In [61]:
# bands from Cumbria, Geopy doesn't detect it
df_new_artists[df_new_artists['origin'].str.contains('Cumbria')]

,artist,origin


In [62]:
df_new_artists['origin'] = df_new_artists['origin'].apply(lambda x: x.replace('Cumbria', 'Cumberland'))

In [63]:
# bands from Middlesex, Geopy doesn't detect it
df_new_artists[df_new_artists['origin'].str.contains('Middlesex')]

,artist,origin


In [64]:
df_new_artists['origin'] = df_new_artists['origin'].apply(lambda x: x.replace(', Middlesex', ''))

In [65]:
# bands from Yorkshire, Geopy doesn't detect it
df_new_artists[df_new_artists['origin'].str.contains('West Riding of Yorkshire')]

,artist,origin


In [66]:
df_new_artists['origin'] = df_new_artists['origin'].apply(lambda x: x.replace(', West Riding of Yorkshire', ''))

In [67]:
# bands from Merseyside, Geopy doesn't detect it
df_new_artists[df_new_artists['origin'].str.contains('Merseyside')]

,artist,origin


In [68]:
df_new_artists['origin'] = df_new_artists['origin'].apply(lambda x: x.replace(', Merseyside', ''))

### **``np.where`` to replace the values for the real origins**

**Try a single origin in Geopy**

In [ ]:
# try to get the right origin of an origin that crashed
geolocator = Nominatim(user_agent="music_analysis")

origin = "Kirkland, Washington"

origin_clean = re.sub(r'\[\d+\]', '', origin).replace('.', '')
location = geolocator.geocode(origin_clean)
print(f"{location.address}")
print(location.latitude, location.longitude)

Kirkland, King County, Washington, United States
47.6765382 -122.2070775


In [212]:
start_index = 70

end_index = start_index+5
df_new_artists[start_index:end_index]

,artist,origin
76,Artifex Pereo,"Louisville, Kentucky, U.S.[1][2]"
77,Midsummer,"Los Angeles, CA, United States"
78,Cold World,"Wilkes-Barre, PA"
79,SQRM,"Springfield, Massachussets"
80,Infest,"Valencia, California, U.S."


**Try all origins in Geopy**

In [8]:
# try to get the coordinates of the origins from Geopy and see if it crashes (wrong location that I have to change)
geolocator = Nominatim(user_agent="music_analysis", timeout=10)

initial_index = 0
count = initial_index-1

for origin in df_new_artists['origin'].str.replace('.', '').str.replace(r'\[\d+\]', '', regex=True)[initial_index:]:
    count+=1
    location = geolocator.geocode(origin)
    print(f"{count} - {location.address}")

0 - Palo Alto, Santa Clara County, California, United States


## **Export to .csv**

### GeoPy wrong locations

**In case GeoPy fails due to a wrong location, I have to delete the new locations, export again, change the location and run GeoPy again**

In [ ]:
#     # import the df with the artists' origins already scraped
# df_artists_origins_scraped = pd.read_csv('Datasets/df_artists_origins.csv')
# df_artists_origins_scraped = df_artists_origins_scraped[0:-20]
# df_artists_origins_scraped.to_csv('Datasets/df_artists_origins.csv', index=False)

In [9]:
df_new_artists.shape

(1, 2)

In [216]:
# in case I created by mistake already 'origin_clean' and I want to drop it
# df_new_artists = df_new_artists[['artist', 'origin']]
# df_new_artists

### **Export to .csv**

In [10]:
export_artists_origins_concat(df_new_artists)

df_artists_origins_concat exported to .csv
(6738, 2)


## **GeoPy**

In [11]:
df_coordinates = get_coordinates_geopy(df_new_artists)
df_coordinates

1/1 - Palo Alto, Santa Clara County, California, United States


,country,city,origin,origin_clean,latitude,longitude,address
0,United States,Palo Alto,"Palo Alto, California, U.S.","Palo Alto, California, US",37.444329,-122.159847,"Palo Alto, Santa Clara County, California, Uni..."


In [12]:
export_coordinates_concat(df_coordinates)

df_coordinates_scraped: (2772, 7)

Found 1 duplicates:
           city        country
2177  Palo Alto  United States

Resulting dataset: (2772, 7)
Merged artists with coordinates! Found 0 new locations
df_coordinates_concat exported to .csv


In [13]:
# merge with the dataframe containing all the artists and their origins
df_artists_origins_coordinates_concat = merge_origins_coordinates(df_new_artists)
df_artists_origins_coordinates_concat.tail()

Exported to a .csv file


,artist,country,city,latitude,longitude,address
6700,Until Your Heart Stops,United States,Oakland,37.804456,-122.271356,"Oakland, Alameda County, California, United St..."
6701,Sleepercar,United States,El Paso,31.760116,-106.487040,"El Paso, El Paso County, Texas, United States"
6702,Swamp Thing,New Zealand / Aotearoa,Rotorua,-38.138149,176.252922,"Rotorua, Rotorua Lakes District, Bay of Plenty..."
6703,Tim Kasher,United States,Omaha,41.258746,-95.938376,"Omaha, Douglas County, Nebraska, United States"
6704,Grateful Dead,United States,Palo Alto,37.444329,-122.159847,"Palo Alto, Santa Clara County, California, Uni..."


## **Statistics**

In [14]:
df_artists_origins_coordinates_concat = pd.read_csv('Datasets/df_artists_origins_coordinates.csv')
df_artists_origins_coordinates_concat.shape

(6705, 6)

In [15]:
brittish_cities = df_artists_origins_coordinates_concat[df_artists_origins_coordinates_concat['country']=='United Kingdom']
american_cities = df_artists_origins_coordinates_concat[df_artists_origins_coordinates_concat['country']=='United States']

In [17]:
order = df_artists_origins_coordinates_concat['country'].value_counts().index
print(df_artists_origins_coordinates_concat['country'].value_counts().head())

# plt.figure(figsize=(8,6))
# sns.countplot(df_artists_origins_coordinates_concat['country'], order=order)

country
United States     4137
United Kingdom    1496
Canada             215
Sverige            156
Australia          133
Name: count, dtype: int64


In [19]:
print(f"{brittish_cities.shape[0]} Brittish artists")
order = brittish_cities['city'].value_counts().index
print(f"{brittish_cities['city'].nunique()} Brittish cities\n")
print(brittish_cities['city'].value_counts().head(15))

# plt.figure(figsize=(9,45))
# sns.countplot(brittish_cities['city'], order=order)

1496 Brittish artists
430 Brittish cities

city
London         402
Birmingham      40
Glasgow         40
Brighton        39
Manchester      39
Leeds           36
Liverpool       30
Bristol         26
Nottingham      22
Sheffield       19
Cardiff         16
Edinburgh       16
Cambridge       15
Reading         13
Southampton     13
Name: count, dtype: int64


In [20]:
print(f"{american_cities.shape[0]} American artists")
order = american_cities['city'].value_counts().index
print(f"{american_cities['city'].nunique()} American cities\n")
print(american_cities['city'].value_counts().head(15))

# plt.figure(figsize=(5,55))
# sns.countplot(df_artists_origins_coordinates_concat[df_artists_origins_coordinates_concat['country']=='United States']['city'], order=order)

4137 American artists
940 American cities

city
Los Angeles      360
New York City    189
Chicago          152
San Francisco    120
Seattle           99
Boston            97
Brooklyn          78
Philadelphia      71
Portland          69
San Diego         65
Atlanta           48
Austin            45
Washington        45
Nashville         41
New York          39
Name: count, dtype: int64


## **``state``**

I want to create a new column to analyze the distribution inside the country

In [21]:
df_artists_origins_coordinates_concat

,artist,country,city,latitude,longitude,address
0,Sparta,United States,El Paso,31.760116,-106.487040,"El Paso, El Paso County, Texas, United States"
1,Muse,United Kingdom,Teignmouth,50.546338,-3.495780,"Teignmouth, Teignbridge, Devon, England, Unite..."
2,Finch,United States,Temecula,33.494635,-117.147366,"Temecula, Riverside County, California, United..."
3,Transplants,United States,Los Angeles,34.053691,-118.242766,"Los Angeles, Los Angeles County, California, U..."
4,Rooney,United States,Los Angeles,34.053691,-118.242766,"Los Angeles, Los Angeles County, California, U..."
...,...,...,...,...,...,...
6700,Until Your Heart Stops,United States,Oakland,37.804456,-122.271356,"Oakland, Alameda County, California, United St..."
6701,Sleepercar,United States,El Paso,31.760116,-106.487040,"El Paso, El Paso County, Texas, United States"
6702,Swamp Thing,New Zealand / Aotearoa,Rotorua,-38.138149,176.252922,"Rotorua, Rotorua Lakes District, Bay of Plenty..."
6703,Tim Kasher,United States,Omaha,41.258746,-95.938376,"Omaha, Douglas County, Nebraska, United States"


In [22]:
df_origins = df_artists_origins_coordinates_concat

In [ ]:
# separate the countries in 2 dataframes
df_artists_uk = df_origins[df_origins['country']=='United Kingdom']
df_artists_us = df_origins[df_origins['country']=='United States']

### **United Kingdom**

In [ ]:
# full address
df_artists_uk.iloc[0]['address']

'Teignmouth, Teignbridge, Devon, England, United Kingdom'

In [ ]:
# I want to get the 3rd element starting from the tail, cause the 2nd one (England for instance) is too generic
df_artists_uk.iloc[0]['address'].split(', ')[-3]

'Devon'

In [42]:
df_artists_uk['address'].apply(lambda x: x.split(', ')[-2])

1                                     England
5                                     England
8                                     England
9                                     England
11                                    England
                        ...                  
6411                                  BN1 1HH
6421                                  England
6575                                  HP1 1EW
6588                                 RM17 6NP
6609    Northern Ireland / Tuaisceart Éireann
Name: address, Length: 1496, dtype: object

In [ ]:
# When I try to do the following it crashes
# df_artists_uk['address'].apply(lambda x: x.split(', ')[-3])

In [ ]:
# create a new column, the length of the string 'address'
df_artists_uk['len_address'] = df_artists_uk['address'].str.len()

# look for short addresses
df_artists_uk.sort_values('len_address').head()

,artist,country,city,latitude,longitude,address,len_address
2974,Anderson Bruford Wakeman Howe,United Kingdom,England,52.531021,-1.264906,"England, United Kingdom",23
982,Pure Reason Revolution,United Kingdom,England,52.531021,-1.264906,"England, United Kingdom",23
2657,Blue Murder,United Kingdom,Inglaterra,52.531021,-1.264906,"England, United Kingdom",23
1162,Yndi Halda,United Kingdom,Kent,51.207075,0.721036,"Kent, England, United Kingdom",29
3654,Graphic Nature,United Kingdom,Kent,51.207075,0.721036,"Kent, England, United Kingdom",29


In [54]:
df_artists_origins

,artist,origin
0,Grateful Dead,"Palo Alto, California, U.S."


### **Change short ``address``**

In [55]:
# import dataframes
df_artists_origins = pd.read_csv('Datasets/df_artists_origins.csv')
df_coordinates = pd.read_csv('Datasets/df_coordinates.csv')
df_artists_origins_coordinates = pd.read_csv('Datasets/df_artists_origins_coordinates.csv')

# print information
print(f'{df_artists_origins.shape}: df_artists_origins')
print(f'{df_coordinates.shape}: df_coordinates')
print(f'{df_artists_origins_coordinates.shape}: df_artists_origins_coordinates')

(6738, 2): df_artists_origins
(2772, 7): df_coordinates
(6705, 6): df_artists_origins_coordinates


In [ ]:
for artist in df_artists_uk.sort_values('len_address').head(3)['artist'].values:
    print(f'df_artists_uk["origin"] = np.where(df_artists_uk["artist"]=="{artist}", "UK", df_artists_uk["origin"])')